In [ ]:
import numpy as np 
import time
import datetime
import os

import pyvisa

import board
import busio
import adafruit_ads1x15.ads1015 as ADS
from adafruit_ads1x15.analog_in import AnalogIn

In [ ]:
'''
    measure temperature using GPIB via an Agilent 82357A USB/GPIB converter
    this converter is hooked up to the raspberry pi and the LakeShore Model 
    218 temperature monitor.
'''

def T_measure():

    return temp

In [ ]:
''' 
    measure gallons per minute using an analog-digital converter
    ADC measures the voltage across a resistive load and converts
    it into a digital signal for the raspberry pi to read in mV's
    ADS1115 documentation at https://docs.circuitpython.org/projects/ads1x15/en/latest/ 
'''
def GPM_measure(resistor_load=1):
    # Create the I2C bus
    i2c = busio.I2C(board.SCL, board.SDA)

    # Create the ADC object using the I2C bus
    ads = ADS.ADS1115(i2c)

    # Create single-ended input on channel 0
    chan = AnalogIn(ads, ADS.P0)

    # Create differential input between channel 0 and 1
    #chan = AnalogIn(ads, ADS.P0, ADS.P1)

    print("{:>5}\t{:>5}".format('raw', 'v'))
    
    ''' perform measurement using ADC and gpib '''
    # use try-except to allow pi to continue working even if error is raised
    try:
        # take measurement and print
        print("{:>5}\t{:>5.3f}".format(chan.value, chan.voltage))
    
    except:
        print("GPM measurement error")
    
    ''' convert measurement '''
    # convert mA measurement to flowrate
    ### range of device is from 4mA to 20mA
    ### gpm is from ASP to AEP, by default 0.00 to 5.28 gpm

    
    #changed SSP and SEP to be function parameters
    #SSP = 4     # signal start point [mV]
    #SEP = 20    # signal end point [mV]
    ASP = 0.00   # analog start point [gpm]
    AEP = 5.28   # analog end point [gpm]

    # shift data down by 4mV (SSP), then 
    # divide by 16mV (SEP-SSP) to get percentage. 
    # then convert to gpm using the meter's analog range
    percentage = (data-SSP)/(SEP-SSP)
    gpm = percentage*(AEP-ASP)

    return gpm

In [ ]:
''' set up measurement devices '''
rm = pyvisa.ResourceManager()
print(rm.list_resources())
my_instrument = rm.open_resource('GPIB0::13::INSTR')
print(my_instrument.query('*IDN?'))

In [ ]:
# this will be appended to the text file

# now save data to opened file
try:
    with open(os.path.join(data_path, filename), 'a') as file:  # use append mode to... append to today's file
        timestamp = str(datetime.datetime.now().strftime("%I:%M:%S %p")) # Hr:Min:Sec am/pm
        data_measurement_string = "[{0}] {1:1.2f} GPM @ {2:1.5f} mV  \n".format(timestamp, flow_rate, data)

        print(data_measurement_string)  # record in console
        file.write(data_measurement_string)  # record in text file

except FileNotFoundError:
    print("except: FileNotFoundError")

In [ ]:
GPM_measure()
